In [1]:
from bs4 import BeautifulSoup as bs
from time import sleep
import requests

### Task 1 Парсинг ваканский с Работа.ру

In [18]:
def site_connection_and_parsing_data(url, headers, params = {}):
    response = requests.get(url=url,headers=headers,params = {})
    status = response.status_code
    
    if not response.ok:
        print(f'Что-то пошло не так. Состояние сервера - {status}')
    else:
        data_parsed = bs(response.content, 'html.parser')
        return data_parsed

    
def parse_vacations(url, headers, page = "1"):
    
    #получение данных с сайта и проверка на валидность состояния ответа
    site_connection_and_parsing_data(url, headers)    
    
    
    # смотрим сколько всего страниц есть и заносим в параметр
    pagination= data_parsed.find_all('a',{'class':'pagination-list__item'})
    max_pagination = int(pagination[-1].text.strip())
    
    
    # задаём итоговый список с результатом
    global vacancies_list
    vacancies_list = []
    
    
    #пробегаемся по каждой странице, чтобы найти вакансии 
    for pages in range(max_pagination):
        
        params = {
        'query':'Программист%20python',
        'sort':'relevance',
        'all_regions':'1',
        'page' : str(pages +1),
        }
        
        site_connection_and_parsing_data(url, headers, params = params)


        # находим вакансии 
        vacancies = data_parsed.find_all('article', {'class':'vacancy-preview-card'})
    
        for vacancy in vacancies:
            #sleep(0.1)
            vacancy_link = 'https://www.rabota.ru' + vacancy.find('a')['href']
            vacancy_name = vacancy.find('a').text.strip()
            vacancy_salary = vacancy.find('div', {'class' : 'vacancy-preview-card__salary'}).find('a').text.replace(u'\xa0', u' ')
            vacancy_description = vacancy.find('div', {'itemprop':'description'}).text
            company = vacancy.find('span', {'class':'vacancy-preview-card__company-name'}).text
            vacancy_location = vacancy.find('span', {'itemprop':'address'}).text.strip()

            vacancy_dict = {
              'name': vacancy_name,
              'link': vacancy_link,
              'salary': vacancy_salary,
              'description': vacancy_description,
              'location': vacancy_location,
          }
            vacancies_list.append(vacancy_dict)
         
    if response.ok:
        #return vacancies_list
        print('Парсинг завершён., результаты сохранены в "vacancies_list"')

In [19]:
url = 'https://www.rabota.ru/vacancy/'
headers = {
    'User_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 YaBrowser/22.11.0.2419 Yowser/2.5 Safari/537.36'
}

parse_vacations(url,headers)

Парсинг завершён., результаты сохранены в "vacancies_list"


In [23]:
print(len(vacancies_list))
vacancies_list

720


[{'name': 'Программист ЧПУ',
  'link': 'https://www.rabota.ru/vacancy/47120301/',
  'salary': 'от 100 000 руб.',
  'description': 'Вакансия компании ООО "ТФИ" Мы занимаемся металлообработкой по заказам клиентов. Заказы как штучные, так и серийные. Благодаря нашему производству работают бесперебойно предприятия, которые обеспечивают людей медициной, едой, теплом, водой. Среди наших клиентов авиация и автоспорт, типографии, буровики…',
  'location': 'Москва'},
 {'name': 'Программист станка с ЧПУ',
  'link': 'https://www.rabota.ru/vacancy/47125127/',
  'salary': '70 000 — 80 000 руб.',
  'description': 'Требуется инженер-технолог на фрезерный станок с ЧПУ. Написание управляющих программ в САМ - системах для станков с ЧПУ. Подбор режимов резания, режущего инструмента. Распил и установка заготовок на станок. Требования: Навыки работы в программе "PowerMill" (приветствуется). Опыт работы на фрезерных…',
  'location': 'Екатеринбург'},
 {'name': 'Программист 1С',
  'link': 'https://www.rabota.